In [1]:
import json
import os
import logging
import numpy as np
import pickle
import keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [2]:
file_path = 'D:\AI Bot\data.json'
with open(file_path) as file:
    data = json.load(file)
logging.info("imported json data to preprocess")    

In [3]:
training_sentence = []
training_labels = list()
labels = []
responses = []

In [4]:
for intent in data['intents']:
    for inputs in intent['input']:
      training_sentence.append(inputs)
      training_labels.append(intent['tag'])
    responses.append(intent['response'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])  
logging.info("Created seperated file for my training sentences and training labels and label and reponses")

num_classes = len(labels)

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)
logging.info("Converted labels into numbers of  training labels")
      

In [5]:
##my unique words in sentences
vocab_size = 1000
##25 will be my  features
embedding_dim = 25
#this for padding max to max 21 words user can put 
max_len = 20

oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_token )
tokenizer.fit_on_texts(training_sentence)

word_index = tokenizer.word_index
sequence = tokenizer.texts_to_sequences(training_sentence)
padded_sequences = pad_sequences(sequence, truncating='post',maxlen=max_len)
#print(padded_sequences)
logging.info("Done Creating Embedding Layer")


In [6]:
logging.info("Stared Creating Model")
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(128, input_shape=(max_len, embedding_dim)))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
logging.info("Compiled the model")

model.summary()
epochs = 70
logging.info("Started training the model")  
model.fit(padded_sequences,np.array(training_labels), epochs=epochs)
    
artifacts_folder = "artifacts"
os.makedirs(artifacts_folder, exist_ok=True)

# Save the trained model to the "artifacts" folder

model_filename = os.path.join(artifacts_folder, 'chat_model.h5')
model.save(model_filename)
logging.info('Model saved to ' + model_filename)

#Save the trained model to the "artifacts" folder

tokenizer_filename = os.path.join(artifacts_folder, 'tokenizer.pickle')
with open(tokenizer_filename,'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"Tokenizer saved to {tokenizer_filename}")
logging.info("Tokenizer saved to {tokenizer_filename}")   

#saving the fitted label encoder
# save the label encoder object to the "artifacts" folder

lbl_encoder_filename = os.path.join(artifacts_folder, 'label_encoder.pickle')
with open(lbl_encoder_filename, 'wb') as enc:
    pickle.dump(lbl_encoder,enc,protocol=pickle.HIGHEST_PROTOCOL)
    print(f"Label Encoder saved to{lbl_encoder_filename}")
logging.info("label saved to{lbl_encoder_filename}")     

C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.0521 - loss: 3.7613
Epoch 2/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0309 - loss: 3.7574   
Epoch 3/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0306 - loss: 3.7500
Epoch 4/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0107 - loss: 3.7408    
Epoch 5/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0631 - loss: 3.7260
Epoch 6/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0465 - loss: 3.7158
Epoch 7/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0373 - loss: 3.6956   
Epoch 8/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0868 - loss: 3.6723
Epoch 9/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0837 - loss: 3.6392
Epoch 10/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0664 - loss: 3.6260
Epoch 11/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.0698 - loss: 3.6302
Epoch 12/70
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0514 - loss

Tokenizer saved to artifacts\tokenizer.pickle
Label Encoder saved toartifacts\label_encoder.pickle


In [2]:
text= 'jayendra yadav'
output=text[:-1]
output    

'jayendra yada'

In [ ]:
import pandas as pd
from pandas import json_normalize

file='D:\AI Bot\data.json'
# Read the JSON data from the file
with open(file_path, 'r') as file:
    data = json.load(file)
    
    
# Convert list elements to strings
for intent in data['intents']:
    intent['input'] = ', '.join(intent['input'])
    intent['response'] = ', '.join(intent['response'])
# Normalize JSON data into DataFrame
df = json_normalize(data['intents'])

# Display DataFrame
df


In [21]:
df

,intents
0,"[{'tag': 'greeting', 'input': ['hello', 'hi', ..."


In [29]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from abc import ABC,abstractclassmethod
from typing import Union
import logging
import neattext as nt
import contractions
from sklearn.preprocessing import LabelEncoder
# Download NLTK resources (do it once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...


In [1]:
def clean_text(text):
        # Initialize NeatText TextFrame and NLTK WordNetLemmatizer
        # Handle contractions
        # Check for empty strings
        '''if not text or len(text) < 2: 
            return text'''
        text = contractions.fix(text)
        docx = nt.TextFrame(text)
        

        # Clean text using NeatText
        cleaned_text = docx.remove_puncts()
        cleaned_text = cleaned_text.remove_special_characters()
        cleaned_text = cleaned_text.remove_userhandles()
        cleaned_text = cleaned_text.remove_urls()
        cleaned_text = cleaned_text.remove_dates()
        cleaned_text = cleaned_text.remove_emails()
        cleaned_text = cleaned_text.remove_emojis()

        # Convert text to lowercase
        cleaned_text = str(cleaned_text).lower()

        # Remove stop words
        stop_words = set(stopwords.words('english'))
        cleaned_text = " ".join(word for word in cleaned_text.split() if word not in stop_words)

        # Lemmatize words
        cleaned_text = " ".join(lemmatizer.lemmatize(word) for word in cleaned_text.split())

        return cleaned_text

In [37]:
df['ninput']=df['input'].apply(clean_text)

In [38]:
df['nresponse']=df['response'].apply(clean_text)

In [ ]:
df